# Training and Testing an MDRNN on Sample Data

This script trains and tests a mixture density recurrent neural network (MDRNN) on 

In [ ]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from context import *
import keras

In [ ]:
# Model Hyperparameters
SEQ_LEN = 100
SEQ_STEP = 1
HIDDEN_UNITS = 128
N_LAYERS = 2
NUMBER_MIXTURES = 5
TIME_DIST = True

# Training Hyperparameters:
BATCH_SIZE = 64
EPOCHS = 100
VAL_SPLIT = 0.10

# Set random seed for reproducibility
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# Fake data:
corpus = empi_mdrnn.generate_data()
display(corpus)

## Test Training

In [ ]:
# Import Human Data CSV and create dt 
perf_df = pd.read_csv('../data/2018-01-25T14-04-35-rnnbox.csv', header=0, index_col=0, parse_dates=['date'])
perf_df['time'] = perf_df.index
perf_df['seconds'] = perf_df.index
perf_df.time = perf_df.time.diff()
perf_df.time = perf_df.time.dt.total_seconds()
perf_df = perf_df.dropna()
perf_df.value = perf_df.value / 255.0
corpus_df = pd.DataFrame({'t': perf_df.time, 'x': perf_df.value})
corpus = np.array(corpus_df)
print("Shape of corpus array:", corpus.shape)
corpus_df.describe()
perf_df.seconds = perf_df.seconds - perf_df.seconds[0]
perf_df.seconds = perf_df.seconds.dt.total_seconds()
perf_df.describe()

In [ ]:
slices = empi_mdrnn.slice_sequence_examples(corpus, SEQ_LEN+1, step_size=SEQ_STEP)

X, y = empi_mdrnn.seq_to_overlapping_format(slices)

X = np.array(X) * empi_mdrnn.SCALE_FACTOR
y = np.array(y) * empi_mdrnn.SCALE_FACTOR

print("Number of training examples:")
print("X:", X.shape)
print("y:", y.shape)

In [ ]:
# Setup Training Model
model = empi_mdrnn.build_model(seq_len=SEQ_LEN, hidden_units=HIDDEN_UNITS, num_mixtures=NUMBER_MIXTURES, layers=2, time_dist=TIME_DIST, inference=False, compile_model=True, print_summary=True)

In [ ]:
# Setup callbacks
model_path = "empi_mdrnn" + "-layers" + str(N_LAYERS) + "-units" + str(HIDDEN_UNITS) + "-mixtures" + str(NUMBER_MIXTURES) + "-scale" + str(empi_mdrnn.SCALE_FACTOR)
filepath = model_path + "-E{epoch:02d}-VL{val_loss:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
terminateOnNaN = keras.callbacks.TerminateOnNaN()
tboard = keras.callbacks.TensorBoard(log_dir='./logs/'+model_path, histogram_freq=2, batch_size=32, write_graph=True, update_freq='epoch')

# Train
history = model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VAL_SPLIT, callbacks=[checkpoint, terminateOnNaN, tboard])
#history = model.fit_generator(generator, steps_per_epoch=300, epochs=100, verbose=1, initial_epoch=0)

# Save final Model
model.save('model_path' + '-final.hdf5')  # creates a HDF5 file of the model

# Test Running Network

In [ ]:
model_file = "empi_mdrnn-layers2-units128-mixtures5-scale10-E55-VL-0.71.hdf5"

#model = empi_mdrnn.build_model(seq_len=SEQ_LEN, hidden_units=HIDDEN_UNITS, num_mixtures=NUMBER_MIXTURES, layers=2, time_dist=TIME_DIST, inference=False, compile_model=True, print_summary=True)
decoder = empi_mdrnn.build_model(seq_len=1, hidden_units=HIDDEN_UNITS, num_mixtures=NUMBER_MIXTURES, layers=2, time_dist=False, inference=True, compile_model=False, print_summary=True)
decoder.load_weights(model_file)

In [ ]:
def generate_sample(model, n_mixtures, prev_sample, temp=1.0, sigma_temp=0.0):
    """Generate prediction for a single touch."""
    out_dim = 2
    params = model.predict(prev_sample.reshape(1,1,out_dim) * SCALE_FACTOR)
    new_sample = mdn.sample_from_output(params[0], out_dim, n_mixtures, temp=temp, sigma_temp=sigma_temp) / SCALE_FACTOR
    new_sample = new_sample.reshape(out_dim,)
    return new_sample

def random_sample():
    return np.array([(0.01 + (np.random.rand()-0.5)*0.005), (np.random.rand()-0.5)*2])

In [ ]:
a = random_sample()
display(a)
display(generate_sample(decoder, NUMBER_MIXTURES, a))

In [ ]:
SCALE_FACTOR = 10

def generate_performance(model, n_mixtures, first_sample, time_limit=5.0, steps_limit=1000, temp=1.0, sigma_temp=0.0):
    """Generates a tiny performance up to 5 seconds in length."""
    out_dim = 2
    time = 0
    steps = 0
    prev_sample = first_sample
    print(prev_sample)
    performance = [prev_sample.reshape((out_dim,))]
    while (steps < steps_limit):  #  and time < time_limit
        params = model.predict(prev_sample.reshape(1,1,out_dim) * SCALE_FACTOR)
        prev_sample = mdn.sample_from_output(params[0], out_dim, n_mixtures, temp=temp, sigma_temp=sigma_temp) / SCALE_FACTOR
        output_touch = prev_sample.reshape(out_dim,)
        # output_touch = constrain_touch(output_touch)
        performance.append(output_touch.reshape((out_dim,)))
        steps += 1
        time += output_touch[1]
    return np.array(performance)

In [ ]:
decoder.reset_states()
mixture_temp = 1.5
sigma_temp = 0.01
t = np.array([(0.01 + (np.random.rand()-0.5)*0.005), (np.random.rand()-0.5)*2]) #robojam.random_touch()
p = generate_performance(decoder, NUMBER_MIXTURES, t, temp=mixture_temp, sigma_temp=sigma_temp, steps_limit=10000)


perf_df = pd.DataFrame({'t':p.T[0], 'x':p.T[1]})
perf_df['time'] = perf_df.t.cumsum()
#plt.show(perf_df.plot('time','x',kind='scatter'))
plt.plot(perf_df.time, perf_df.x, '.r-')
plt.show()
print(perf_df.describe())

## Investigate Output
window = 100
for n in [1000,2000,3000,4000,5000,6000]:
    print("Window:", str(n),'to',str(n+window))
    plt.plot(perf_df[n:n+window].time, perf_df[n:n+window].x, '.r-')
    plt.show()